In [1]:
import pandas as pd
import os
import sqlite3 as db
from scipy.stats import ttest_1samp
import numpy as np

In [2]:
'''
Read data from database
'''
path = r'../BacktestResults'
directory=os.fsencode(path)

backtest_results=[]

query='SELECT * FROM Details'

for file in os.listdir(directory):
    filename=os.fsdecode(file)
    if filename.endswith(".db"):
        db_file=path+"\\"+filename
        conn=db.connect(db_file)
        result=pd.read_sql(query, conn)
        conn.close()
        backtest_results.append(result)
        print(f'retrieved results from \'{filename}\'')
        
all_results=pd.concat(backtest_results, axis=0)

all_results['Label']=all_results['TrainStart']+'_'+all_results['StocksLabel'].apply(lambda x: ''.join(x))
all_results=all_results[~((all_results['Method']=='MVP') & (all_results['AggregateMethod']=='geo_median'))]
all_results['Gamma2']=all_results['Gamma2'].fillna(0.001)
all_results.loc[all_results['Method']=='MVP', 'AggregateMethod']='NA'
all_results['AggregateMethod']=all_results['AggregateMethod'].fillna('sample_mean')
all_results=all_results[all_results['Gamma2']==0.001]
all_results=all_results[all_results['N_Day_Rebalance']==1]


all_results['MaximumDrawdown']=np.abs(all_results['MaximumDrawdown'])

all_results=all_results[['Method', 'AggregateMethod', 'Label' ,'Return', 'SharpeRatio', 'Volatility', 'TurnoverRate', 'MaximumDrawdown']].copy()



all_results['Method']=all_results['Method'].str.replace('EMVP', 'BOOT_SSR_MVP')

all_results

retrieved results from 'BS_SSR_Geo.db'
retrieved results from 'BS_SSR_Geo_2.db'
retrieved results from 'Geo_SSR_Boot_emvp.db'
retrieved results from 'mean_mvp_boot_ssr_emvp.db'
retrieved results from 'SSR_BOOT_MVP_Mean.db'


,Method,AggregateMethod,Label,Return,SharpeRatio,Volatility,TurnoverRate,MaximumDrawdown
0,BOOT_SSR_MVP,geo_median,"2019-01-23_['HRL', 'CSCO', 'PAYX', 'CNP', 'DGX...",-0.035395,0.064530,0.338384,0.344300,0.363882
1,BOOT_SSR_MVP,geo_median,"2018-02-17_['MRO', 'AXON', 'LH', 'MMC', 'CRM',...",0.308217,1.776944,0.158973,0.337399,0.063346
2,BOOT_SSR_MVP,geo_median,"2017-12-08_['SNA', 'LOW', 'AMZN', 'C', 'TFX', ...",0.139518,0.841938,0.173668,0.376247,0.141987
3,BOOT_SSR_MVP,geo_median,"2012-01-30_['COF', 'ALB', 'HIG', 'SYY', 'IFF',...",0.312353,1.355589,0.218255,0.281093,0.125890
4,BOOT_SSR_MVP,geo_median,"2019-11-14_['CPT', 'HRL', 'GD', 'CRM', 'GIS', ...",0.122702,0.697152,0.193672,0.340691,0.125450
...,...,...,...,...,...,...,...,...
45,SSR_BOOT_MVP,sample_mean,"2013-04-05_['KDP', 'V', 'BSX', 'JCI', 'KO', 'C...",0.271946,1.371048,0.190152,0.305514,0.165578
46,SSR_BOOT_MVP,sample_mean,"2014-01-26_['L', 'BBY', 'BKR', 'AMT', 'IBM', '...",-0.133800,-0.667940,0.188335,0.309449,0.213870
47,SSR_BOOT_MVP,sample_mean,"2016-02-20_['CSCO', 'SBUX', 'MU', 'CRM', 'EOG'...",0.280605,1.629891,0.160361,0.321592,0.115659
48,SSR_BOOT_MVP,sample_mean,"2020-09-30_['MET', 'ITW', 'ADP', 'MDT', 'VRSN'...",-0.133051,-0.428121,0.256199,0.328673,0.282242


In [5]:
all_results['Label'].unique().shape

(50,)

In [6]:
for method in all_results['Method'].unique():
    shape=all_results[all_results['Method']==method].shape
    
    print(f'{method} shape: {shape}')

BOOT_SSR_MVP shape: (100, 8)
SSR_BOOT_MVP shape: (100, 8)
Boot_mvp shape: (100, 8)
SSR_mvp shape: (100, 8)
MVP shape: (50, 8)


In [7]:
geo_median_results=all_results[all_results['AggregateMethod'].isin(['geo_median', 'NA'])]
geo_median_results

,Method,AggregateMethod,Label,Return,SharpeRatio,Volatility,TurnoverRate,MaximumDrawdown
0,BOOT_SSR_MVP,geo_median,"2019-01-23_['HRL', 'CSCO', 'PAYX', 'CNP', 'DGX...",-0.035395,0.064530,0.338384,0.344300,0.363882
1,BOOT_SSR_MVP,geo_median,"2018-02-17_['MRO', 'AXON', 'LH', 'MMC', 'CRM',...",0.308217,1.776944,0.158973,0.337399,0.063346
2,BOOT_SSR_MVP,geo_median,"2017-12-08_['SNA', 'LOW', 'AMZN', 'C', 'TFX', ...",0.139518,0.841938,0.173668,0.376247,0.141987
3,BOOT_SSR_MVP,geo_median,"2012-01-30_['COF', 'ALB', 'HIG', 'SYY', 'IFF',...",0.312353,1.355589,0.218255,0.281093,0.125890
4,BOOT_SSR_MVP,geo_median,"2019-11-14_['CPT', 'HRL', 'GD', 'CRM', 'GIS', ...",0.122702,0.697152,0.193672,0.340691,0.125450
...,...,...,...,...,...,...,...,...
45,MVP,NA,"2013-04-05_['KDP', 'V', 'BSX', 'JCI', 'KO', 'C...",0.559687,1.246041,0.435931,0.224899,0.324850
46,MVP,NA,"2014-01-26_['L', 'BBY', 'BKR', 'AMT', 'IBM', '...",-0.395570,-0.972726,0.424838,0.104659,0.483180
47,MVP,NA,"2016-02-20_['CSCO', 'SBUX', 'MU', 'CRM', 'EOG'...",0.475261,1.228325,0.375881,0.104001,0.317253
48,MVP,NA,"2020-09-30_['MET', 'ITW', 'ADP', 'MDT', 'VRSN'...",-0.165965,-0.142068,0.475265,0.247018,0.350709


In [8]:
simple_mean_results=all_results[all_results['AggregateMethod'].isin(['sample_mean', 'NA'])]
simple_mean_results

,Method,AggregateMethod,Label,Return,SharpeRatio,Volatility,TurnoverRate,MaximumDrawdown
0,MVP,NA,"2019-01-23_['HRL', 'CSCO', 'PAYX', 'CNP', 'DGX...",0.031874,0.306953,0.479983,0.209749,0.355935
1,MVP,NA,"2018-02-17_['MRO', 'AXON', 'LH', 'MMC', 'CRM',...",0.517273,1.248201,0.395948,0.088122,0.200648
2,MVP,NA,"2017-12-08_['SNA', 'LOW', 'AMZN', 'C', 'TFX', ...",-0.026181,0.080016,0.324220,0.231183,0.290620
3,MVP,NA,"2012-01-30_['COF', 'ALB', 'HIG', 'SYY', 'IFF',...",0.023054,0.255468,0.388477,0.182496,0.286159
4,MVP,NA,"2019-11-14_['CPT', 'HRL', 'GD', 'CRM', 'GIS', ...",-0.051711,0.005817,0.331533,0.155547,0.238505
...,...,...,...,...,...,...,...,...
45,SSR_BOOT_MVP,sample_mean,"2013-04-05_['KDP', 'V', 'BSX', 'JCI', 'KO', 'C...",0.271946,1.371048,0.190152,0.305514,0.165578
46,SSR_BOOT_MVP,sample_mean,"2014-01-26_['L', 'BBY', 'BKR', 'AMT', 'IBM', '...",-0.133800,-0.667940,0.188335,0.309449,0.213870
47,SSR_BOOT_MVP,sample_mean,"2016-02-20_['CSCO', 'SBUX', 'MU', 'CRM', 'EOG'...",0.280605,1.629891,0.160361,0.321592,0.115659
48,SSR_BOOT_MVP,sample_mean,"2020-09-30_['MET', 'ITW', 'ADP', 'MDT', 'VRSN'...",-0.133051,-0.428121,0.256199,0.328673,0.282242


In [10]:
simple_mean_results

,Method,AggregateMethod,Label,Return,SharpeRatio,Volatility,TurnoverRate,MaximumDrawdown
0,MVP,NA,"2019-01-23_['HRL', 'CSCO', 'PAYX', 'CNP', 'DGX...",0.031874,0.306953,0.479983,0.209749,0.355935
1,MVP,NA,"2018-02-17_['MRO', 'AXON', 'LH', 'MMC', 'CRM',...",0.517273,1.248201,0.395948,0.088122,0.200648
2,MVP,NA,"2017-12-08_['SNA', 'LOW', 'AMZN', 'C', 'TFX', ...",-0.026181,0.080016,0.324220,0.231183,0.290620
3,MVP,NA,"2012-01-30_['COF', 'ALB', 'HIG', 'SYY', 'IFF',...",0.023054,0.255468,0.388477,0.182496,0.286159
4,MVP,NA,"2019-11-14_['CPT', 'HRL', 'GD', 'CRM', 'GIS', ...",-0.051711,0.005817,0.331533,0.155547,0.238505
...,...,...,...,...,...,...,...,...
45,SSR_BOOT_MVP,sample_mean,"2013-04-05_['KDP', 'V', 'BSX', 'JCI', 'KO', 'C...",0.271946,1.371048,0.190152,0.305514,0.165578
46,SSR_BOOT_MVP,sample_mean,"2014-01-26_['L', 'BBY', 'BKR', 'AMT', 'IBM', '...",-0.133800,-0.667940,0.188335,0.309449,0.213870
47,SSR_BOOT_MVP,sample_mean,"2016-02-20_['CSCO', 'SBUX', 'MU', 'CRM', 'EOG'...",0.280605,1.629891,0.160361,0.321592,0.115659
48,SSR_BOOT_MVP,sample_mean,"2020-09-30_['MET', 'ITW', 'ADP', 'MDT', 'VRSN'...",-0.133051,-0.428121,0.256199,0.328673,0.282242


In [8]:
def hypothesis_testing(df, metric, base='MVP', sequence=['MVP', 'SSR_mvp', 'Boot_mvp', 'SSR_BOOT_MVP', 'BOOT_SSR_MVP']):
    metric_df=df[['Method', 'Label', metric]].set_index([ 'Label','Method']).unstack()
    metric_df.columns=[col[1] for col in metric_df.columns]
    metric_df=metric_df[sequence]
    compare_baseline=metric_df.copy()

    for strategy in metric_df.columns:
        compare_baseline[strategy]=compare_baseline[strategy]-metric_df[base]
            
    if metric in ['Return', 'SharpeRatio']:
        ttest=compare_baseline.apply(lambda x: ttest_1samp(x, popmean=0, alternative='greater'), axis=0)
        
    else:
        ttest=compare_baseline.apply(lambda x: ttest_1samp(x, popmean=0, alternative='less'), axis=0)
    
    ttest.index=['t-statistic', 'p-value']
    return ttest


def hypothesis_testing_other_strat_as_base(df, metric, target='MVP', sequence=['MVP', 'SSR_mvp', 'Boot_mvp', 'SSR_BOOT_MVP', 'BOOT_SSR_MVP']):
    metric_df=df[['Method', 'Label', metric]].set_index([ 'Label','Method']).unstack()
    metric_df.columns=[col[1] for col in metric_df.columns]
    metric_df=metric_df[sequence]
    compare_baseline=metric_df.copy()

    for strategy in metric_df.columns:
        compare_baseline[strategy]=metric_df[target]-metric_df[strategy]
            
    if metric in ['Return', 'SharpeRatio']:
        ttest=compare_baseline.apply(lambda x: ttest_1samp(x, popmean=0, alternative='greater'), axis=0)
        
    else:
        ttest=compare_baseline.apply(lambda x: ttest_1samp(x, popmean=0, alternative='less'), axis=0)
    
    ttest.index=['t-statistic', 'p-value']
    return ttest

def hypothesis_testing_geo_vs_simple(simple_df, geo_df, metric,  sequence=['MVP', 'SSR_mvp', 'Boot_mvp', 'SSR_BOOT_MVP', 'BOOT_SSR_MVP']):
    simple_metric_df=simple_df[['Method', 'Label', metric]].set_index([ 'Label','Method']).unstack()
    simple_metric_df.columns=[col[1] for col in simple_metric_df.columns]
    simple_metric_df=simple_metric_df[sequence]
    
    geo_metric_df=geo_df[['Method', 'Label', metric]].set_index([ 'Label','Method']).unstack()
    geo_metric_df.columns=[col[1] for col in geo_metric_df.columns]
    geo_metric_df=geo_metric_df[sequence]
    
    compare_baseline=geo_metric_df-simple_metric_df

    #for strategy in metric_df.columns:
    #    compare_baseline[strategy]=metric_df[target]-metric_df[strategy]
            
    if metric in ['Return', 'SharpeRatio']:
        ttest=compare_baseline.apply(lambda x: ttest_1samp(x, popmean=0, alternative='greater'), axis=0)
        
    else:
        ttest=compare_baseline.apply(lambda x: ttest_1samp(x, popmean=0, alternative='less'), axis=0)
    
    ttest.index=['t-statistic', 'p-value']
    return ttest


In [9]:
def test_all_metrics(df, base='MVP', all_metrics=['Return', 'SharpeRatio', 'Volatility', 'TurnoverRate', 'MaximumDrawdown']):
    t_statistics_list=[]
    p_value_list=[]
    
    for metric in all_metrics:
        ttest=hypothesis_testing(df, metric, base)
        t_stat=ttest.loc[['t-statistic']]
        t_stat.index=[metric]
        t_statistics_list.append(t_stat)
        
        p_value=ttest.loc[['p-value']]
        p_value.index=[metric]
        p_value_list.append(p_value)
        
    all_t_stats=pd.concat(t_statistics_list, axis=0)
    all_p_values=pd.concat(p_value_list, axis=0)
    
    return all_t_stats, all_p_values

def test_all_metrics_other_strat_as_base(df, target='MVP', all_metrics=['Return', 'SharpeRatio', 'Volatility', 'TurnoverRate', 'MaximumDrawdown']):
    t_statistics_list=[]
    p_value_list=[]
    
    for metric in all_metrics:
        ttest=hypothesis_testing_other_strat_as_base(df, metric, target)
        t_stat=ttest.loc[['t-statistic']]
        t_stat.index=[metric]
        t_statistics_list.append(t_stat)
        
        p_value=ttest.loc[['p-value']]
        p_value.index=[metric]
        p_value_list.append(p_value)
        
    all_t_stats=pd.concat(t_statistics_list, axis=0)
    all_p_values=pd.concat(p_value_list, axis=0)
    
    return all_t_stats, all_p_values

def test_all_metrics_simple_vs_geo(simple_df=simple_mean_results, geo_df=geo_median_results, all_metrics=['Return', 'SharpeRatio', 'Volatility', 'TurnoverRate', 'MaximumDrawdown']):
    t_statistics_list=[]
    p_value_list=[]
    
    for metric in all_metrics:
        ttest=hypothesis_testing_geo_vs_simple(simple_df, geo_df,metric)
        t_stat=ttest.loc[['t-statistic']]
        t_stat.index=[metric]
        t_statistics_list.append(t_stat)
        
        p_value=ttest.loc[['p-value']]
        p_value.index=[metric]
        p_value_list.append(p_value)
        
    all_t_stats=pd.concat(t_statistics_list, axis=0)
    all_p_values=pd.concat(p_value_list, axis=0)
    
    return all_t_stats, all_p_values

In [10]:
simple_mean_t_stat, simple_mean_p_val=test_all_metrics(simple_mean_results)

In [11]:
simple_mean_p_val

,MVP,SSR_mvp,Boot_mvp,SSR_BOOT_MVP,BOOT_SSR_MVP
Return,NaN,4.603144e-02,2.919117e-03,4.514456e-02,3.912892e-02
SharpeRatio,NaN,1.261473e-04,3.350200e-05,1.006757e-05,1.224991e-05
Volatility,NaN,3.149383e-27,1.547903e-20,1.069472e-29,2.104913e-28
TurnoverRate,NaN,1.000000e+00,2.327711e-07,1.000000e+00,1.000000e+00
MaximumDrawdown,NaN,1.190332e-17,8.914212e-13,3.943057e-19,1.763292e-18


In [12]:
geo_median_t_stat, geo_median_p_val=test_all_metrics(geo_median_results)
geo_median_p_val

,MVP,SSR_mvp,Boot_mvp,SSR_BOOT_MVP,BOOT_SSR_MVP
Return,NaN,7.192340e-02,5.733275e-03,1.794103e-02,3.134142e-02
SharpeRatio,NaN,4.267650e-03,4.213596e-04,1.628958e-05,1.794325e-05
Volatility,NaN,3.069191e-26,3.033242e-16,1.166855e-29,3.128944e-30
TurnoverRate,NaN,1.000000e+00,4.725754e-10,1.000000e+00,1.000000e+00
MaximumDrawdown,NaN,3.955518e-15,8.210187e-11,5.070122e-18,3.090653e-19


In [13]:
BS_vs_ensemble_t_stat, BS_vs_ensemble_p_val=test_all_metrics(simple_mean_results, base='Boot_mvp')
BS_vs_ensemble_p_val

,MVP,SSR_mvp,Boot_mvp,SSR_BOOT_MVP,BOOT_SSR_MVP
Return,0.997081,0.985242,NaN,9.835495e-01,9.755635e-01
SharpeRatio,0.999966,0.984630,NaN,8.272842e-01,7.645949e-01
Volatility,1.000000,0.000162,NaN,2.294993e-12,7.425787e-12
TurnoverRate,1.000000,1.000000,NaN,1.000000e+00,1.000000e+00
MaximumDrawdown,1.000000,0.312344,NaN,9.800785e-06,1.810711e-05


In [14]:
Boot_simple_mean_t_stat, Boot_simple_mean_p_val=test_all_metrics_other_strat_as_base(simple_mean_results, target='Boot_mvp')
Boot_simple_mean_p_val

,MVP,SSR_mvp,Boot_mvp,SSR_BOOT_MVP,BOOT_SSR_MVP
Return,2.919117e-03,1.475805e-02,NaN,1.645051e-02,2.443652e-02
SharpeRatio,3.350200e-05,1.537029e-02,NaN,1.727158e-01,2.354051e-01
Volatility,1.547903e-20,9.998375e-01,NaN,1.000000e+00,1.000000e+00
TurnoverRate,2.327711e-07,1.917880e-53,NaN,4.811913e-45,8.776108e-45
MaximumDrawdown,8.914212e-13,6.876562e-01,NaN,9.999902e-01,9.999819e-01


In [15]:
Boot_geo_median_t_stat, Boot_geo_median_p_val=test_all_metrics_other_strat_as_base(geo_median_results, target='Boot_mvp')
Boot_geo_median_p_val

,MVP,SSR_mvp,Boot_mvp,SSR_BOOT_MVP,BOOT_SSR_MVP
Return,5.733275e-03,3.109683e-02,NaN,1.239040e-01,7.634258e-02
SharpeRatio,4.213596e-04,1.749931e-02,NaN,3.274434e-01,2.947725e-01
Volatility,3.033242e-16,8.834763e-01,NaN,9.999998e-01,1.000000e+00
TurnoverRate,4.725754e-10,2.383265e-49,NaN,3.269564e-45,6.538204e-42
MaximumDrawdown,8.210187e-11,1.921541e-01,NaN,9.985509e-01,9.998974e-01


In [16]:
simple_v_geo_t_stat, simple_v_geo_p_value=test_all_metrics_simple_vs_geo()

In [17]:
simple_v_geo_p_value

,MVP,SSR_mvp,Boot_mvp,SSR_BOOT_MVP,BOOT_SSR_MVP
Return,NaN,0.942677,7.772464e-01,0.113728,0.715987
SharpeRatio,NaN,0.999991,9.769725e-01,0.884751,0.991086
Volatility,NaN,1.000000,9.988606e-01,1.000000,1.000000
TurnoverRate,NaN,1.000000,8.501562e-14,1.000000,0.083270
MaximumDrawdown,NaN,1.000000,9.984749e-01,1.000000,0.999995


In [18]:
geo_v_simple_t_stat, geo_v_simple_p_value=test_all_metrics_simple_vs_geo(simple_df=geo_median_results, geo_df=simple_mean_results)

In [19]:
geo_v_simple_p_value

,MVP,SSR_mvp,Boot_mvp,SSR_BOOT_MVP,BOOT_SSR_MVP
Return,NaN,5.732271e-02,0.222754,8.862723e-01,2.840132e-01
SharpeRatio,NaN,8.672151e-06,0.023028,1.152488e-01,8.913995e-03
Volatility,NaN,7.006295e-18,0.001139,1.227247e-16,5.427315e-09
TurnoverRate,NaN,4.422926e-34,1.000000,6.302068e-31,9.167295e-01
MaximumDrawdown,NaN,1.848444e-10,0.001525,9.933044e-08,4.974065e-06


In [20]:
export_path='hypothesis_testing_results.xlsx'

with pd.ExcelWriter(export_path) as writer:
    simple_mean_t_stat.to_excel(writer, sheet_name='Simple T-stat MVP Base')
    geo_median_t_stat.to_excel(writer, sheet_name='Geo T-stat MVP Base')
    
    simple_mean_p_val.to_excel(writer, sheet_name='Simple P-Val MVP Base')
    geo_median_p_val.to_excel(writer, sheet_name='Geo P-Val MVP Base')
    
    Boot_simple_mean_t_stat.to_excel(writer, sheet_name='Simple T-stat Boot Target')
    Boot_geo_median_t_stat.to_excel(writer, sheet_name='Geo T-stat Boot Target')
       
    Boot_simple_mean_p_val.to_excel(writer, sheet_name='Simple P-Val Boot Target')
    Boot_geo_median_p_val.to_excel(writer, sheet_name='Geo P-Val Boot Target')
    
    BS_vs_ensemble_t_stat.to_excel(writer, sheet_name='Simple T-stat Boot Base')
    BS_vs_ensemble_p_val.to_excel(writer, sheet_name='Simple P-Val Boot Base')
    
    geo_v_simple_t_stat.to_excel(writer, sheet_name='Geo base v Simple T-stat')
    simple_v_geo_t_stat.to_excel(writer, sheet_name='Simple base v Geo T-stat')
    
    geo_v_simple_p_value.to_excel(writer, sheet_name='Geo base v Simple P-val')
    simple_v_geo_p_value.to_excel(writer, sheet_name='Simple base v Geo P-val')
    
    